# Исследование надежности заемщиков.

Проект: Исследование надёжности заёмщиков
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

Открываем таблицу и выводим информацию о получееном файле на экран.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Выводим часть таблицы для изучения полученной информации.

In [3]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


###  Шаг 2. Заполнение пропусков

Поиск пропущенных значений, нашли пропуски в 2х столбцах days_employed и total_income.

В столбцах days_employed и total_income нашли 2174 пропусков. Всего 21525 записей. Что составляет 10% от всех записей.

Колличество значений с пропусками в столбцах идентично. Оба столбца в формате float64. 
Возможно, допущены тех. ошибки при выгрузке.

Возможно, кто-то поленился или постестнялся указать, посчитав пункты необязательными.

In [4]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Заменяем пропуски в столбцах соответствующими медианными значениями.

Так как в столбцах есть и отрицательные и положительные значения, использование среднего значения будет некорректно.

Лучшим в такой ситуации будет использование медианного значения. 

Находим медиану по столбцу с пропусками days_employed - трудовой стаж

In [5]:
days_employed_median=data['days_employed'].median()
print('Медиана по столбцу с пропусками days_employed',days_employed_median)
total_income_median=data['total_income'].median()

Медиана по столбцу с пропусками days_employed -1203.369528770489


Находим медиану по столбцу с пропусками total_income - общий доход

In [6]:
print('Медиана по столбцу с пропусками total_income',total_income_median)
data['days_employed']=data['days_employed'].fillna(days_employed_median)
data['total_income']=data['total_income'].fillna(total_income_median)

Медиана по столбцу с пропусками total_income 145017.93753253992


Проверяем, остались ли пропуски.

In [7]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропусков в данных не осталось. 

### Шаг 2.2 Проверка данных на аномалии и исправления.

В данных столбцах есть числа. Выведем минимальные значения данных столбцв, чтобы исключить отрицательные значения в данных.
children, days_employed, dob_years, education_id, family_status_id, debt, total_income. Создадим отдельный список количесвенных параметров.

In [8]:
column_count=['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'debt', 'total_income']

In [9]:
for column in data[column_count]:
    print(f'Минимальное значение в столбце {column}',data[column].min())

Минимальное значение в столбце children -1
Минимальное значение в столбце days_employed -18388.949900568383
Минимальное значение в столбце dob_years 0
Минимальное значение в столбце education_id 0
Минимальное значение в столбце family_status_id 0
Минимальное значение в столбце debt 0
Минимальное значение в столбце total_income 20667.26379327158


Считаем кол-во отрицательных значений в столбце days_employed. 

In [10]:
data['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

Считаем кол-во отрицательных значений в столбце days_employed. 

In [11]:
data.loc[data['days_employed']<0]['days_employed'].count()

18080

Смотрим значения в столбце children и их колличество.

In [12]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце children 76 строк со значением 20. Это явна опечатка, вероятно, что детей 2,а не 20. Заменим 20 на 2. И проверим значения в столбце children. 

In [13]:
data.loc[data['children']==20] = 2
data['children'].value_counts()

 0    14149
 1     4818
 2     2131
 3      330
-1       47
 4       41
 5        9
Name: children, dtype: int64

Все отрицательные значения в столбцах days_employed и children заменим на положительные. Проверим как сработала замена, узнаем минимальные значения в соответствующих столбцах.

In [14]:
data['days_employed'] = abs(data['days_employed'])
data['children'] = abs(data['children'])
print('Минимальное значение в столбце children',data['children'].min())
print('Минимальное значение в столбце days_employed',data['days_employed'].min())

Минимальное значение в столбце children 0
Минимальное значение в столбце days_employed 2.0


Отрицательных значений в столбцах не осталось.

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце total_income и days_employed на целочисленный.

In [15]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')

Проверим, прошла ли замена типа.

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Проверим, есть ли в данных полные дубликаты. 

In [17]:
data.duplicated().sum()

129

Удаляем дубликаты. Проверяем их колличество после удаления. 

In [18]:
data=data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

Проверяем наличие неявных дубликатов. В столбцах со строками: education,family_status, gender, income_type, purpose.            

В столбце education.

In [19]:
data['education'].value_counts()

среднее                13653
высшее                  4698
СРЕДНЕЕ                  770
Среднее                  705
неоконченное высшее      666
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
2                          1
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Меняем значения, сводя все к нижнему регистру. И заново считаем уникальные значения в столбце education.

In [20]:
data['education']=data['education'].str.lower()
data['education'].value_counts()

среднее                15128
высшее                  5237
неоконченное высшее      742
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверяем уникальные значения в столбце family_status.

In [21]:
data['family_status'].value_counts()

женат / замужем          12295
гражданский брак          4151
Не женат / не замужем     2801
в разводе                 1193
вдовец / вдова             955
2                            1
Name: family_status, dtype: int64

Дублей нет. Но сведем все к нижнему регистру. И проверим выводом кол-ва уникальных значений.

In [22]:
data['family_status']=data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12295
гражданский брак          4151
не женат / не замужем     2801
в разводе                 1193
вдовец / вдова             955
Name: family_status, dtype: int64

Проверяем уникальные значения в столбце gender.

In [23]:
data['gender'].value_counts()

F      14142
M       7252
2          1
XNA        1
Name: gender, dtype: int64

Выведем на экран строку таблицы с указанным значением XNA в столбце gender.

In [24]:
data.loc[data['gender']=='XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10662,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


По указанным данным определить пол клиента не удастся. Так как у нас лишь 1 строка с таким значением, а всего строк 21471. Удалим ее. 

In [25]:
data=data[data['gender']!='XNA']
data['gender'].value_counts()

F    14142
M     7252
2        1
Name: gender, dtype: int64

Проверяем уникальные значения в столбце income_type.

In [26]:
data['income_type'].value_counts()

сотрудник          11048
компаньон           5057
пенсионер           3828
госслужащий         1455
предприниматель        2
безработный            2
2                      1
студент                1
в декрете              1
Name: income_type, dtype: int64

In [27]:
data[data['income_type']==2]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,2,2,2,NaN,2,NaN,2,2,2,2,2,2


Данная строка явно ошибочная. Удалим ее из DF. И проверим значения столбца income_type еще раз.

In [28]:
data=data[data['income_type']!=2]
data['income_type'].value_counts()

сотрудник          11048
компаньон           5057
пенсионер           3828
госслужащий         1455
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создаем два новых датафрейма, в которых: 1. каждому уникальному значению из education соответствует уникальное значение education_id — в первом;2.каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.
Используя индексацию проверяем соответствие education_id - education. И family_status_id - family_status.
data.loc[data['education_id']==0]
4 - ученая степень
3 - начальное образование(школа)
2 - неоконченное высшее
1 - среднее
0 - высшее
data.loc[data['family_status_id']==0]
4 - не женат / не замужем	
3 - в разводе	
2 - вдовец / вдова	
1 - гражданский брак	
0 - женат / замужем	

In [29]:
data_education = data[['education','education_id']]
data_education.head(10)
data_education['education_id'].value_counts()

1    15128
0     5237
2      741
3      282
4        6
Name: education_id, dtype: int64

In [30]:
data_family_status = data[['family_status','family_status_id']]
data_family_status.head(10)
data_family_status['family_status_id'].value_counts()

0    12295
1     4150
4     2801
3     1193
2      955
Name: family_status_id, dtype: int64

Удаляем из исходного датафрейма столбцы education и family_status.

In [31]:
data_final=data[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose']]
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21394 entries, 0 to 21395
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21394 non-null  int64 
 1   days_employed     21394 non-null  int64 
 2   dob_years         21394 non-null  int64 
 3   education_id      21394 non-null  int64 
 4   family_status_id  21394 non-null  int64 
 5   gender            21394 non-null  object
 6   income_type       21394 non-null  object
 7   debt              21394 non-null  int64 
 8   total_income      21394 non-null  int64 
 9   purpose           21394 non-null  object
dtypes: int64(7), object(3)
memory usage: 1.8+ MB


### Шаг 2.6. Категоризация дохода.

Пишем функцию для проведения категоризации.

In [32]:
def income_group(income):
    if income in range(0,30000):
        return 'E'
    elif (income in range(30000,50000)):
        return 'D'
    elif (income in range (50000,200000)):
        return 'C'
    elif (income in range(200000,1000000)):
        return 'B'
    else: 
        return 'A'

6.	На основании диапазонов, указанных ниже, создайте столбец total_income_category с категориями:
o	0–30000 — 'E';
o	30001–50000 — 'D';
o	50001–200000 — 'C';
o	200001–1000000 — 'B';
o	1000001 и выше — 'A'.


In [33]:
data_final['total_income_category'] = data_final['total_income'].apply(income_group)
data_final.head()

/tmp/ipykernel_53/3627892889.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['total_income_category'] = data_final['total_income'].apply(income_group)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


In [34]:
data_final['total_income_category'].value_counts()

C    15977
B     5020
D      350
A       25
E       22
Name: total_income_category, dtype: int64

In [35]:
data_final['total_income_category'].value_counts().sum()

21394

In [36]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21394 entries, 0 to 21395
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21394 non-null  int64 
 1   days_employed          21394 non-null  int64 
 2   dob_years              21394 non-null  int64 
 3   education_id           21394 non-null  int64 
 4   family_status_id       21394 non-null  int64 
 5   gender                 21394 non-null  object
 6   income_type            21394 non-null  object
 7   debt                   21394 non-null  int64 
 8   total_income           21394 non-null  int64 
 9   purpose                21394 non-null  object
 10  total_income_category  21394 non-null  object
dtypes: int64(7), object(4)
memory usage: 2.0+ MB


Сверили кол-во строк в столбце total_income_category и в файле с данными. Совпадают. Ничего не потеряли.

### Шаг 2.7. Категоризация целей кредита.

7.	Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
o	'операции с автомобилем',
o	'операции с недвижимостью',
o	'проведение свадьбы',
o	'получение образования'.
Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.
Вы можете использовать собственную функцию и метод apply(). Изучите данные в столбце purpose и определите, какие подстроки помогут вам правильно определить категорию


In [37]:
def purpose_group(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif ('недвиж') in purpose:
        return 'операции с недвижимостью'
    elif ('жил') in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образ' in purpose:
        return 'получение образования'          

In [38]:
data_final['purpose_category'] = data_final['purpose'].apply(purpose_group)
data_final['purpose_category'].value_counts()

операции с недвижимостью    10777
операции с автомобилем       4292
получение образования        3999
проведение свадьбы           2326
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок? Сделаем сводную таблицу по зависимости колличества детей от колличества задолженностей по кредиту. 

In [39]:
data_pivot_children_debt = data_final.pivot_table('debt', 'children', aggfunc=['mean','count'])
data_pivot_children_debt

,mean,count
,debt,debt
children,,
0,0.075358,14106
1,0.091639,4856
2,0.094542,2052
3,0.081818,330
4,0.097561,41
5,0.000000,9


##### Вывод 1:

Получается, что доля должников без детей ниже, чем среди людей с детьми.
В группах с 1,2 и 4 детьми доля должников примерно одинаковая.
При этом доля должников в группах с 3мя детьми несколько ниже чем с 1,2 или 4мя.
Выводы по группе с 5тью детьми делать совсем не разумно,у нас таких 9. Явно мало для выводов.
Получается. что надежнее одобрять кредит бездетным людям.

##### Вопрос 2:

Есть ли заивисимость между семейным положением и возвратом кредита в срок? Сделаем сводную таблицу по зависимости семейного положения от колличества задолженностей по кредиту.

In [40]:
data_pivot_marriage_debt = data.pivot_table(index = ['family_status_id','family_status'], values='debt', aggfunc=['mean','count'])
data_pivot_marriage_debt

,,mean,count
,,debt,debt
family_status_id,family_status,,
0,женат / замужем,0.075478,12295
1,гражданский брак,0.092771,4150
2,вдовец / вдова,0.065969,955
3,в разводе,0.070411,1193
4,не женат / не замужем,0.097465,2801


##### Вывод 2:

Комментарий студента:
Получается, что самая большая доля должников в категории не женат/не замужем.
Те, кто живут в гражданском браке реже задолжают.
Наименьшие доли должников в категориях вдовец/вдова и в разводе. Доля должников для тех, кто живет в браке выше чем в категориях вдовец/вдова и в разводе, но ниже тех, кто живет гражданским браком или не женам.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок? Сделаем сводную таблицу по зависимости дохода от колличества случаев возникнования задолженностей по кредиту.

In [41]:
data_pivot_income_debt = data_final.pivot_table(index = ['total_income_category'], values='debt', aggfunc=['mean','count'])
data_pivot_income_debt

,mean,count
,debt,debt
total_income_category,,
A,0.080000,25
B,0.070518,5020
C,0.084747,15977
D,0.060000,350
E,0.090909,22


##### Вывод 3:

Получается, что самая большая доля должников наблюдается в категирии Е(<30000). Наименьшая в категории D(<50000). Категории с доходом(до 200000) и с самым высоки доходом схожи по кол-ву должников. При этом категория с доходом до 1000000 немного реже становится должниками.
<br>Четкой зависимости дохода от уровня дохода нет. Она то повышается, то понижается. 
<br>Изходя и данных нашей выборки наилучшими клиентами с мин долей долников являются люди с доходом <50 000 руб.


##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок? Сделаем сводную таблицу по целям кредита от колличества случаев возникнования задолженностей по кредиту.

In [42]:
data_pivot_purpose_debt = data_final.pivot_table(index = ['purpose_category'], values='debt', aggfunc=['mean','count'])
data_pivot_purpose_debt

,mean,count
,debt,debt
purpose_category,,
операции с автомобилем,0.093430,4292
операции с недвижимостью,0.072376,10777
получение образования,0.092273,3999
проведение свадьбы,0.078676,2326


##### Вывод 4:

Получается, что самая большая доля должников в категории операции с автомобилем, примерно там же кредиты на образование.
<br>Меньше всего доля должников у тех, кто решал вопросы с недвижимостью или для организации свадьбы.
<br>Таким образом, самыми надежными являются лица, которые обращаются за кредитом на операции с недвижимостью.

## Общий вывод:

<br>Согласно полученым данным, значительное влияние могут оказать семейное положение, колличество детей, а также цель кредитования. 
<br>Тогда как, доход клиента не всегда будет четко свидетельствовать о возможности погашения в срок. 
<br>На мой взгялд, лучшим клиентом(согласно изученным данным) будет бездетный человек, в браке и с доходом до 50 000 руб. 
Целью кредитования - операции с недвижимостью.